In [1]:
input_dim = 4
num_clusters = 4

ntrials = 10
lg = 1.5 #lambda_global

import numpy as np
import os
from sklearn.cluster import KMeans

import sys
sys.path.insert(0, "../Code/")
from myplot import plot_groups, plot_metrics
from train_multiclass import train_multiclass

old_stdout = sys.stdout

In [2]:
# Configure the data generation process and train the autoencoder

def gen(n = 400):
    
    x = np.zeros((n, 4))
    y = np.zeros((n))
    
    for i in range(n):
        
        if np.random.uniform() < 0.5:
            x[i, 0] = 1.0 + np.random.normal(loc = 0.0, scale = 0.2)
            y[i] += 1
            
        if np.random.uniform() < 0.5:
            x[i, 1] = 1.0 + np.random.normal(loc = 0.0, scale = 0.2)
            y[i] += 2
            
        x[i, 2] = np.random.normal(loc = 0.0, scale = 0.5)
        
        x[i, 3] = x[i, 0] + np.random.normal(loc = 0.0, scale = 0.05)
        
    return x, y

x, y = gen()

y = y.astype(int)

os.system("rm -rf Model")

sess, rep, prob, grad_i, X, D, Y, I = train_multiclass(x, y, 4)

sys.stdout = old_stdout





The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





INFO:tensorflow:Summary name Cross Entropy is illegal; using Cross_Entropy instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Summary name Recon MSE is illegal; using Recon_MSE instead.







INFO:tensorflow:Restoring parameters from ./model.cpkt


In [3]:
indices = []
for i in range(num_clusters):
    v = []
    for j in range(x.shape[0]):
        if y[j] == i:
            v.append(j)
    indices.append(v)


In [4]:
def info(x, i):
    p, g = sess.run([prob, grad_i], feed_dict = {X: x, I: i, D: np.zeros((1, input_dim))})
    return p, g[0]

sys.path.insert(0, "../Integrated-Gradients/IntegratedGradients/")
from integrated_gradients import integrated_gradients

def ig(init, target, avg_class = False, num_classes = num_clusters, normalize = True):
    average = np.zeros((input_dim))
    count = 0
    for index_ref in indices[init]:
        baseline = x[index_ref, :]
        for index_target in indices[target]:
            point = x[index_target, :]
            if avg_class:
                for target_class in range(num_classes):
                    ig, _ = integrated_gradients(point, target_class, info, baseline)
                    average += ig
                    count += 1
            else:
                ig, _ = integrated_gradients(point, target, info, baseline)
                average += ig
                count += 1
    average /= count
    if normalize:
        average /= np.sum(np.abs(average))
        average = np.round(average, decimals = 3)        
    return average 

deltas = []
for g in range(1, num_clusters):
    deltas.append(ig(0, g))
    
deltas = np.array(deltas)
np.save("deltas_ig.npy", deltas)


print("IG consistently uses the SC variable")
print(deltas)


IG consistently uses the SC variable
[[ 0.538  0.     0.04   0.422]
 [ 0.     0.97   0.026 -0.004]
 [ 0.506  0.173  0.037  0.284]]


In [5]:
print("IG is not always symmetrical")

print(ig(1, 2))
print(ig(2, 1))

print(ig(1, 2, avg_class = True, normalize = False))
print(ig(2, 1, avg_class = True, normalize = False))



IG is not always symmetrical
[0.644 0.09  0.027 0.239]
[ 0.1    0.822 -0.027  0.05 ]
[-5.43663697e-09  1.27770000e-08 -3.80158532e-10  9.67926288e-10]
[ 1.10543752e-08 -1.76557505e-09  3.72075646e-10  6.91986807e-10]


In [6]:

i1 = indices[1][3]
i2 = indices[2][0]

x1 = x[i1, :]
t1 = y[i1]
x2 = x[i2, :]
t2 = y[i2]

print("Class 1 ", t1,  " Class 2 ", t2)

print("IG is symmetrical when the `class of interest' is held constant")

print("Setting it to ", t1)
v, _ = integrated_gradients(x1, t1, info, x2)
print(v)
v, _ = integrated_gradients(x2, t1, info, x1)
print(v)
print("Setting it to ", t2)
v, _ = integrated_gradients(x1, t2, info, x2)
print(v)
v, _ = integrated_gradients(x2, t2, info, x1)
print(v)
print("But it isn't as soon as you ask `why is this point in its assigned class as opposed to that other point?'")

o = 0
for i in range(num_clusters):
    print("Target ", i)
    v, _ = integrated_gradients(x1, i, info, x2)
    print(v)
    o += v
o /= num_clusters
print("Averaged: ", o)

o = 0
for i in range(num_clusters):
    print("Target ", i)
    v, _ = integrated_gradients(x2, i, info, x1)
    print(v)
    o += v
o /= num_clusters
print("Averaged: ", o)


Class 1  1  Class 2  2
IG is symmetrical when the `class of interest' is held constant
Setting it to  1
[ 0.09537793  0.99651434 -0.10363884  0.02365101]
[-0.09537793 -0.99651456  0.10363884 -0.02365101]
Setting it to  2
[-0.67494852 -0.20911251  0.01573494 -0.14763854]
[ 0.67494852  0.20911254 -0.01573494  0.14763857]
But it isn't as soon as you ask `why is this point in its assigned class as opposed to that other point?'
Target  0
[-3.13398207e-05  2.42678309e-05  2.10337435e-05 -1.61275033e-05]
Target  1
[ 0.09537793  0.99651434 -0.10363884  0.02365101]
Target  2
[-0.67494852 -0.20911251  0.01573494 -0.14763854]
Target  3
[ 0.57960197 -0.78742603  0.08788282  0.12400371]
Averaged:  [ 1.13022957e-08  1.65290679e-08 -1.02628226e-08  1.25745784e-08]
Target  0
[ 3.13398207e-05 -2.42678258e-05 -2.10337408e-05  1.61275086e-05]
Target  1
[-0.09537793 -0.99651456  0.10363884 -0.02365101]
Target  2
[ 0.67494852  0.20911254 -0.01573494  0.14763857]
Target  3
[-0.57960192  0.7874258  -0.087882